In [2]:
import json, os, glob, math, sys
sys.path.append('../')
from constants import costanalyzer
from pymatgen.core.structure import Composition
from pymatgen.entries.computed_entries import ComputedStructureEntry
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from criterias import filter_selectivity
from samuel_TOF import get_distance_TOF, select_N2
from constants import activity_params, Eads_ref_rpbe, Eads_ref_oc20, Ncorr_db, Ocorr_db
from other_functions import str_to_hkl
import numpy as np

def hkl_to_latex(hkl, hkil=False):
    hkl_as_str = []
    for h in hkl:
        if h < 0:
            hkl_as_str.append('\\bar{%s}' %(abs(h)))
        else:
            hkl_as_str.append(str(h))
            
    if hkil:
        i = -1*(hkl[0]+hkl[1])
        if i < 0:
            i = '\\bar{%s}' %(abs(i))
        miller_index = '(%s %s %s %s)' %(hkl_as_str[0], hkl_as_str[1], i, hkl_as_str[2])
    else:
        miller_index = '(%s %s %s)' %tuple(hkl_as_str)
        
    return miller_index
        
def analyze_selectivity_for_binary(EadsO, EadsN):

    distances, N2_selectivity = [], []
    for v in [0, 0.1, 0.2]:
        for act in ['excellent', 'good']:
            pt2 = activity_params[v][act]['pt2']
            pt1 = activity_params[v][act]['pt1']
            a = activity_params[v][act]['a']
            b = activity_params[v][act]['b']
            c = activity_params[v][act]['c']
            dist = get_distance_TOF(EadsO, EadsN, a, b, c, pt1, pt2)
            distances.append(dist) 
            N2_selectivity.append(bool(select_N2([EadsO, EadsN])))
            
    return distances, N2_selectivity

In [3]:
final_entries_01032022 = json.load(open('../datasets/final_entries_01032022.json', 'rb'))
final_entries_01032022 = sorted(final_entries_01032022, key=lambda d:
                                costanalyzer.get_cost_per_kg(Composition(d['composition'])))
best_eads = json.load(open('../datasets/best_01262022.json'))

In [5]:
final_candidates = {'aflow:23523de3fe98621d': 'Cu3Ni','aflow:51c8542a7ce5f16e': 'Cu3Ni',
                    'aflow:0174436f419f3b03': 'Cu3Ni','aflow:ce386e28a5836f2b': 'Cu4Ni',
                    'aflow:02c206bf38a574ae': 'Cu5Ni','aflow:27d4f10811c294e8': 'Cu5Ni',
                    'mp-54': 'Co','mp-79': 'Zn','aflow:87c1fae46433ffc9': 'Cu2Ag',
                    'aflow:02be216bddab7f56': 'Cu2Ag','aflow:8784858a1a4968bf': 'Cu3Ag',
                    'aflow:f842a63ecfd96380': 'Cu3Ag','aflow:feb23a8bc99256f2': 'Cu4Ag',
                    'aflow:faa99f7ba2865909': 'Cu5Ag4','aflow:dc47422a610dc3bf': 'Cu2Ag',
                    'aflow:eec6cee145cf1470': 'Cu2Ag','aflow:abccb4a3861a7e14': 'Cu3Ag',
                    'aflow:af6c89431239b7ef': 'Cu3Ag','aflow:b6cef1af578df09e': 'Cu3Ag',
                    'aflow:d5c01982f9e55e52': 'Cu3Ag','aflow:fed26b81230e899c': 'Cu3Ag',
                    'aflow:3076eaae893c7514': 'Cu3Ag','mp-23': 'Ni',
                    'aflow:cfdda3512dadc2ee': 'CoCu7', 'aflow:bdf8f32f44428a49': 'ZnCu8'}

final_entries = []
for entry_id in final_candidates.keys():
    pair = tuple(sorted(Composition(final_candidates[entry_id]).as_dict().keys()))
    fname = '%s_%s.json' %pair if len(pair) == 2 else '%s.json' %pair
    entries = [d for d in json.load(open(os.path.join(fdir, fname), 'rb'))]
    final_entries.extend([entry for entry in entries if entry['entry_id'] == entry_id])

valid_mpids = [d['entry_id'] for d in final_entries]
tier4_dict, tier4_pairs, hatch_dict, entry_id_to_hatch_dict = \
filter_selectivity(valid_mpids, return_entry_id_to_hatch_dict=True, fdir=fdir+'*')

In [6]:
final_entries = sorted(final_entries, key=lambda d: costanalyzer.get_cost_per_kg(d['composition']))

for d in final_entries:
    if d['entry_id'] in ['aflow:abccb4a3861a7e14', 'aflow:fed26b81230e899c']:
        continue
    entry = ComputedStructureEntry.from_dict(d)
    formula = entry.composition.reduced_formula
    sg = SpacegroupAnalyzer(entry.structure).get_space_group_symbol()
    if '-' in sg:
        i = sg.index('-')
        sg = sg.replace(sg[i]+sg[i+1], '\\bar{%s}' %(sg[i+1]))
            
    cost = costanalyzer.get_cost_per_kg(formula)
    if len(entry.composition.as_dict().keys()) == 2:
        lowest_g = {float(v): min([g[2] for g in d['data']['G_pbx'] if '%.1f' %(g[0]) == v]) for v in ['0.0', '0.1']}
    else:
        lowest_g = {0.0: 0, 0.1: 0}
    lowest_dist_ex = {float(v): min([vals[0] for vals in 
                                     d['data']['activity_selectivity'][v]['excellent'].values()]) for v in ['0', '0.1']}
    lowest_dist_good = {float(v): min([vals[0] for vals in 
                                       d['data']['activity_selectivity'][v]['good'].values()]) for v in ['0', '0.1']}
    lowest_dist = {v: min([lowest_dist_ex[v], lowest_dist_good[v]]) for v in [0, 0.1]}
    vcolor = {v: 'green' if lowest_dist[v] < 0.31 else 'red' for v in lowest_dist.keys()}
    select = {0: 'both', 1: '\ce{N2}', 2: 'both'}
    counts = [entry_id_to_hatch_dict[entry.entry_id].count('xx'), 
              entry_id_to_hatch_dict[entry.entry_id].count('\\\\'),
              entry_id_to_hatch_dict[entry.entry_id].count('////')]
    active_at_01 = 'Yes' if vcolor[0.1] == 'green' else 'No'
    
    if select[counts.index(max(counts))] == 'both':
        print('\ce{%s} & $%s$ & %.2f & %.2f & %.2f & %s & %s & %s\\\\' \
              %(formula, sg, cost, lowest_g[0.0],
                lowest_g[0.1], active_at_01, '\\checkmark', '\\checkmark'))
    else:
        print('\ce{%s} & $%s$ & %.2f & %.2f & %.2f & %s & %s & %s\\\\' \
              %(formula, sg, cost, lowest_g[0.0],
                lowest_g[0.1], active_at_01, '\\checkmark', ''))


\ce{Zn} & $P6_3/mmc$ & 2.95 & 0.00 & 0.00 & No & \checkmark & \checkmark\\
\ce{ZnCu8} & $I4/mmm$ & 8.78 & 0.18 & 0.29 & No & \checkmark & \checkmark\\
\ce{Cu5Ni} & $Cm$ & 10.93 & 0.15 & 0.27 & Yes & \checkmark & \checkmark\\
\ce{Cu5Ni} & $Amm2$ & 10.93 & 0.15 & 0.27 & Yes & \checkmark & \checkmark\\
\ce{Cu4Ni} & $I4/m$ & 11.22 & 0.17 & 0.29 & Yes & \checkmark & \checkmark\\
\ce{Cu3Ni} & $R\bar{3}m$ & 11.65 & 0.20 & 0.33 & Yes & \checkmark & \checkmark\\
\ce{Cu3Ni} & $I4/mmm$ & 11.65 & 0.21 & 0.33 & Yes & \checkmark & \checkmark\\
\ce{Cu3Ni} & $Cmmm$ & 11.65 & 0.21 & 0.33 & Yes & \checkmark & \checkmark\\
\ce{CoCu7} & $Fm\bar{3}m$ & 14.52 & 0.19 & 0.30 & No & \checkmark & \checkmark\\
\ce{Ni} & $Fm\bar{3}m$ & 18.54 & 0.00 & 0.00 & Yes & \checkmark & \checkmark\\
\ce{Co} & $P6_3/mmc$ & 52.21 & 0.00 & 0.00 & No & \checkmark & \\
\ce{Cu4Ag} & $I4/m$ & 261.58 & 0.08 & 0.16 & No & \checkmark & \checkmark\\
\ce{Cu3Ag} & $P4/mmm$ & 315.24 & 0.08 & 0.15 & Yes & \checkmark & \checkmark\\
\ce{Cu3

In [7]:
good_hkls = {}
for entry_id in best_eads.keys():
    if entry_id not in good_hkls.keys():
        good_hkls[entry_id] = []
    for hkl in best_eads[entry_id].keys():        
    
        O_list, N_list = best_eads[entry_id][hkl]['O'], best_eads[entry_id][hkl]['N']
        O_list = sorted(O_list, key=lambda e: e['eads'])
        EadsO = (O_list[0]['eads'] + Eads_ref_oc20['*O'] - Eads_ref_rpbe['*O'])*Ocorr_db[1]+Ocorr_db[0]
        N_list = sorted(N_list, key=lambda e: e['eads'])
        EadsN = (N_list[0]['eads'] + Eads_ref_oc20['*N'] - Eads_ref_rpbe['*N'])*Ncorr_db[1]+Ncorr_db[0]
        distances, N2_selectivity = analyze_selectivity_for_binary(EadsO, EadsN)
        if any([d < 0.31 for d in distances]) and any(N2_selectivity):
            good_hkls[entry_id].append(hkl)

In [8]:
final_hkls = {}
for formula_entry_id in good_hkls.keys():
    formula, entry_id = formula_entry_id.split('_')
    final_hkls[entry_id] = [hkl for hkl in good_hkls[formula_entry_id] if 1 
                            < max([abs(i) for i in str_to_hkl(hkl)])]
#     if len(final_hkls[entry_id]) > 12:
#         natoms, hkls = zip(*sorted(zip(good_hkls_natoms[formula_entry_id], final_hkls[entry_id])))
#         final_hkls[entry_id] = hkls[:12]
    
for d in final_entries:
    entry = ComputedStructureEntry.from_dict(d)
    entry_id = entry.entry_id
    if entry_id not in final_hkls:
        continue
    crystal = SpacegroupAnalyzer(entry.structure).get_crystal_system()
    final_hkls[entry.entry_id] = [hkl_to_latex(str_to_hkl(hkl), 
                                               hkil=crystal in ['hexagonal', 'rhombohedral']) 
                                  for hkl in final_hkls[entry_id]]

In [9]:
final_entries = sorted(final_entries, key=lambda d: costanalyzer.get_cost_per_kg(d['composition']))
count = 0
for d in final_entries:
    entry = ComputedStructureEntry.from_dict(d)
    if entry.entry_id not in final_hkls.keys():
        continue
    if not final_hkls[entry.entry_id]:
        continue
    formula = entry.composition.reduced_formula
    sg = SpacegroupAnalyzer(entry.structure).get_space_group_symbol()
    if '-' in sg:
        i = sg.index('-')
        sg = sg.replace(sg[i]+sg[i+1], '\\bar{%s}' %(sg[i+1]))

    cost = costanalyzer.get_cost_per_kg(formula)
    if len(entry.composition.as_dict().keys()) == 2:
        lowest_g = {float(v): min([g[2] for g in d['data']['G_pbx'] if '%.1f' %(g[0]) == v]) for v in ['0.0', '0.1']}
    else:
        lowest_g = {0.0: 0, 0.1: 0}
    lowest_dist_ex = {float(v): min([vals[0] for vals in 
                                     d['data']['activity_selectivity'][v]['excellent'].values()]) for v in ['0', '0.1']}
    lowest_dist_good = {float(v): min([vals[0] for vals in 
                                       d['data']['activity_selectivity'][v]['good'].values()]) for v in ['0', '0.1']}
    lowest_dist = {v: min([lowest_dist_ex[v], lowest_dist_good[v]]) for v in [0, 0.1]}
    vcolor = {v: 'green' if lowest_dist[v] < 0.31 else 'red' for v in lowest_dist.keys()}
    select = {0: 'both', 1: '\ce{N2}', 2: 'both'}
    counts = [entry_id_to_hatch_dict[entry.entry_id].count('xx'), 
              entry_id_to_hatch_dict[entry.entry_id].count('\\\\'),
              entry_id_to_hatch_dict[entry.entry_id].count('////')]
    active_at_01 = 'Yes' if vcolor[0.1] == 'green' else 'No'
    
    hkls = final_hkls[entry.entry_id]
    split_hkls = np.array_split(hkls, int(math.ceil(len(hkls)/6)))
    for i, hkl_list in enumerate(split_hkls):
        if len(hkl_list) < 2:
            continue
        if i == 0:
            print('\ce{%s} & $%s$ & $%s$ \\\\' %(formula, sg, ', '.join(hkl_list)))
        else:
            print('& & $%s$ \\\\' %(', '.join(hkl_list)))
    print('\\hline')
    count+=1
#     print('\ce{%s} & $%s$ & $%s$ \\\\' %(formula, sg, ', '.join(final_hkls[entry.entry_id])))



\hline
\hline
\ce{Cu5Ni} & $Cm$ & $(2 1 1), (4 2 \bar{3}), (2 1 \bar{2}), (4 1 2), (2 0 \bar{3}), (1 0 \bar{2})$ \\
& & $(1 3 4), (3 3 \bar{4}), (3 2 3), (3 2 0), (3 1 2), (3 1 \bar{2})$ \\
& & $(3 1 \bar{3}), (3 0 \bar{1}), (3 0 \bar{2}), (1 0 \bar{3}), (2 1 2), (4 2 3)$ \\
& & $(4 0 \bar{1}), (4 0 \bar{3}), (3 3 4), (1 3 \bar{4}), (1 2 4)$ \\
& & $(1 2 3), (1 2 \bar{3}), (1 1 \bar{4}), (1 0 3), (2 3 4)$ \\
& & $(2 3 \bar{4}), (2 2 3), (2 2 \bar{3}), (2 1 \bar{3}), (2 1 \bar{4})$ \\
\hline
\ce{Cu5Ni} & $Amm2$ & $(0 4 1), (0 3 1), (0 1 2), (0 2 3)$ \\
& & $(0 1 3), (1 2 4), (1 1 4)$ \\
\hline
\ce{Cu4Ni} & $I4/m$ & $(2 2 1), (3 2 0), (3 1 0), (4 3 0)$ \\
& & $(2 1 2), (4 1 0), (3 3 4)$ \\
\hline
\ce{Cu3Ni} & $R\bar{3}m$ & $(2 1 2), (2 1 0), (2 1 \bar{2}), (4 1 4), (4 1 0)$ \\
& & $(4 1 \bar{4}), (4 \bar{1} 4), (4 \bar{1} 0), (1 0 2)$ \\
& & $(1 0 4), (1 0 3), (1 0 \bar{3}), (1 0 \bar{4})$ \\
\hline
\ce{Cu3Ni} & $I4/mmm$ & $(1 0 3), (4 3 0), (2 1 0), (4 1 4), (4 1 0)$ \\
& & $(3 2 0), (3